# 🎙️ ASR + Tradução com **Microsoft Phi‑4‑multimodal‑instruct** no Google Colab — *Versão Colab‑safe*

Esta versão ajusta as dependências para evitar conflitos em ambientes Google Colab com GPU **Tesla T4** (Driver CUDA 12.x).
- Remove **torchvision** (não é necessário para áudio) e fixa **Pillow** em `10.3.0` (evita conflito com Gradio).
- Não tenta instalar **flash-attn** (T4 não suporta); o notebook usa `attn_implementation='eager'`.

Restante funcionalidade: igual à versão original (ASR PT/ES/EN, fallback de tradução para Bengali, Cloudflared).


## 1) Setup do ambiente (GPU, libs, FFmpeg, Cloudflared) — Colab‑safe

In [ ]:
# Verifica GPU disponível
!nvidia-smi || true

# Instala dependências SEM upgrade (para não sobrescrever Pillow depois)
!pip install -q --no-warn-conflicts transformers==4.46.0 accelerate==1.3.0 soundfile==0.13.1 backoff==2.2.1 peft==0.13.2 gradio==4.44.1 jedi markupsafe==2.0.1

# CRÍTICO: Força Pillow 10.3.0 DEPOIS de todas as outras deps (evita erro 'is_directory' em PIL._util)
!pip uninstall -y pillow Pillow PIL 2>/dev/null || true
!pip install --force-reinstall --no-cache-dir --no-deps pillow==10.3.0

# FFmpeg para processamento de áudio
!apt-get -qq update && apt-get -qq install -y ffmpeg >/dev/null 2>&1

# Instala Cloudflared (TryCloudflare)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared && chmod +x cloudflared

# Verifica versão do Pillow instalada
import PIL
print(f"✅ Setup completo - Pillow {PIL.__version__}")

## 2) Login no Hugging Face com token
Guarda o token em **Colab → Secrets** com o nome `HF_TOKEN`. Se não estiver presente, será aberto um prompt para inserir manualmente.

In [ ]:
from huggingface_hub import login
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    if hf_token:
        login(hf_token)
        print('✅ Login feito ao Hugging Face via Colab Secrets.')
    else:
        login()
        print('✅ Login feito ao Hugging Face via prompt.')
except Exception as e:
    print('Falha no login:', e)


## 3) Carregar o modelo **Phi‑4‑multimodal‑instruct** e o Processor (usar atenção *eager*)

In [ ]:
import os
os.environ["HF_USE_FLASH_ATTENTION_2"] = "0"  # força desativação de FlashAttention2

import torch, numpy as np, soundfile as sf
from transformers import AutoModelForCausalLM, AutoProcessor, AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM
import types
import sys

# Detecta GPU e configura dtype apropriado
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
USE_GPU = DEVICE == 'cuda'
DTYPE = torch.float16 if USE_GPU else torch.float32
print(f"🔧 Device: {DEVICE.upper()}, dtype: {DTYPE}")

# Verifica versões compatíveis
import PIL
print(f"📦 Pillow versão: {PIL.__version__} (esperado: 10.3.0)")
if PIL.__version__ != "10.3.0":
    print("⚠️ AVISO: Versão de Pillow incompatível. Se encontrares erro 'is_directory', volta à célula 1 e executa novamente.")

# Configura SDPA apenas se tiver CUDA (CPU não tem flash sdp)
if USE_GPU:
    try:
        torch.backends.cuda.enable_flash_sdp(False)
        torch.backends.cuda.enable_mem_efficient_sdp(False)
        torch.backends.cuda.enable_math_sdp(True)
        print("🔧 SDPA configurado: flash=False, mem_efficient=False, math=True")
    except Exception:
        pass

MODEL_ID = 'microsoft/Phi-4-multimodal-instruct'

# Cache global para M2M tokenizer/model (evita reloads)
_M2M_CACHE = {'tokenizer': None, 'model': None}

def get_m2m_models():
    """Carrega M2M tokenizer/model uma vez e reutiliza."""
    if _M2M_CACHE['tokenizer'] is None:
        m2m_id = 'facebook/m2m100_418M'
        _M2M_CACHE['tokenizer'] = AutoTokenizer.from_pretrained(m2m_id)
        _M2M_CACHE['model'] = AutoModelForSeq2SeqLM.from_pretrained(m2m_id).to(DEVICE)
    return _M2M_CACHE['tokenizer'], _M2M_CACHE['model']

# Load processor
print(f"Carregando processor para {MODEL_ID}...") 
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
print("✅ Processor carregado.")

# Carrega a configuração e força attn_implementation='eager'
config = AutoConfig.from_pretrained(MODEL_ID, trust_remote_code=True)
config.attn_implementation = 'eager'
print(f"🔧 Configuração de atenção forçada para: {config.attn_implementation}")

# --- PATCH PRE-MODEL-LOAD: Injeta prepare_inputs_for_generation na classe antes de instanciar ---
print("🔧 Preparando patch de prepare_inputs_for_generation na classe Phi4MMModel...")

# Definição do método que vai ser adicionado
def prepare_inputs_for_generation(self, input_ids, **kwargs):
    """Prepara inputs para geração. Delega ao modelo interno se disponível."""
    # Tenta delegar ao modelo interno (LlamaForCausalLM) se existir
    if hasattr(self, 'model') and hasattr(self.model, 'prepare_inputs_for_generation'):
        return self.model.prepare_inputs_for_generation(input_ids, **kwargs)
    
    # Se for um wrapper PEFT (LoraModel), tenta acessar a base_model
    if hasattr(self, 'base_model') and hasattr(self.base_model, 'prepare_inputs_for_generation'):
        return self.base_model.prepare_inputs_for_generation(input_ids, **kwargs)
    
    # Fallback: retorna um dict com os inputs principais
    model_inputs = {"input_ids": input_ids}
    for key in ["attention_mask", "position_ids", "past_key_values", "use_cache", "inputs_embeds",
                "pixel_values", "image_embeds", "image_attention_mask"]:
        if key in kwargs:
            model_inputs[key] = kwargs[key]
    return model_inputs

# Monkey-patch a classe na cache de módulos ANTES do from_pretrained
for module_name, module in list(sys.modules.items()):
    if 'modeling_phi4mm' in module_name and hasattr(module, 'Phi4MMModel'):
        print(f"🔧 Encontrado módulo {module_name}, adicionando prepare_inputs_for_generation à classe...")
        if not hasattr(module.Phi4MMModel, 'prepare_inputs_for_generation'):
            module.Phi4MMModel.prepare_inputs_for_generation = prepare_inputs_for_generation
            print(f"✅ Método adicionado à classe Phi4MMModel em {module_name}")
        break

# Carrega o modelo com eager attention, float16 para T4
print(f"Carregando modelo {MODEL_ID}... Isto pode levar alguns minutos.")
try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        config=config,
        device_map='auto',
        offload_folder="/tmp/",
        torch_dtype=DTYPE,
        trust_remote_code=True,
        attn_implementation='eager',
    )
    print("✅ Modelo carregado com sucesso.")
except Exception as e:
    print(f"🔧 Detectado erro de prepare_inputs_for_generation durante load, reparando...")
    # Procura novamente e patcheia
    for module_name, module in list(sys.modules.items()):
        if 'modeling_phi4mm' in module_name and hasattr(module, 'Phi4MMModel'):
            module.Phi4MMModel.prepare_inputs_for_generation = prepare_inputs_for_generation
            print(f"✅ Método adicionado à classe (segunda tentativa)")
            break
    # Tenta novamente
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        config=config,
        device_map='auto',
        offload_folder="/tmp/",
        torch_dtype=DTYPE,
        trust_remote_code=True,
        attn_implementation='eager',
    )

# Garante que a configuração interna fica em 'eager' e desativa cache
try:
    if hasattr(model, 'config'):
        model.config.attn_implementation = 'eager'
        model.config.use_cache = False
except Exception:
    pass

# Adapter de SPEECH (áudio) - confirmado que existe no modelo
print('🔧 Carregando adapter "speech-lora"...')
try:
    model.load_adapter(MODEL_ID, adapter_name='speech', adapter_kwargs={'subfolder': 'speech-lora', 'offload_folder': '/tmp/'})
    model.set_adapter('speech')
    print('✅ Adapter "speech-lora" carregado e ativado com sucesso.')
except Exception as e:
    print(f'⚠️ AVISO: speech-lora não disponível: {e}')
    print('   Continua com modelo base (qualidade pode ser reduzida)')

# Injeção final: adiciona o método à instância se ainda não existir
if not hasattr(model, 'prepare_inputs_for_generation'):
    print("🔧 Adicionando método à instância do modelo...")
    model.prepare_inputs_for_generation = types.MethodType(prepare_inputs_for_generation, model)
    print("✅ Método adicionado à instância.")
else:
    print("✅ Método já presente no modelo.")

# Garante que generation_config também tem use_cache=False
try:
    if hasattr(model, 'generation_config'):
        model.generation_config.use_cache = False
except Exception:
    pass

print(f"\n✅ Modelo pronto para ASR! Device: {DEVICE.upper()}, dtype: {DTYPE}")


## 4) Funções de ASR e tradução

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time, os, traceback

SUPPORTED_SPEECH = {
    'português': 'Portuguese',
    'espanhol': 'Spanish',
    'inglês': 'English',
}


def _safe_decode_text(processor, gen_ids):
    try:
        return processor.batch_decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    except Exception:
        # Fallback via tokenizer, se existir
        try:
            tok = getattr(processor, 'tokenizer', None)
            if tok is not None:
                return tok.batch_decode(gen_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        except Exception:
            pass
        # Último recurso: tentar converter ids em string básica
        try:
            return str(gen_ids)
        except Exception:
            return "<decode_failed>"


def phi4_transcribe_or_translate(audio_path: str, in_lang_ui: str, out_lang_ui: str) -> str:
    try:
        t0 = time.perf_counter()
        # Carrega áudio
        audio_array, sr = sf.read(audio_path)
        # Converte para mono se necessário
        if audio_array.ndim > 1:
            audio_array = audio_array.mean(axis=1)
        # Limita duração para evitar demora excessiva
        max_seconds = 30
        max_samples = int(sr * max_seconds)
        if audio_array.shape[0] > max_samples:
            audio_array = audio_array[:max_samples]
        audio_array = np.asarray(audio_array, dtype=np.float32)

        # Instrução base (sem placeholder de áudio)
        if out_lang_ui.lower() == "bengali":
            base_instruction = f"Transcribe the audio clip into text in {SUPPORTED_SPEECH.get(in_lang_ui, 'Portuguese')}."
        else:
            if in_lang_ui == out_lang_ui:
                base_instruction = f"Transcribe the audio clip into text in {SUPPORTED_SPEECH.get(in_lang_ui, 'Portuguese')}."
            else:
                base_instruction = (
                    f"Transcribe the audio to text in {SUPPORTED_SPEECH.get(in_lang_ui, 'Portuguese')}, "
                    f"and then translate the audio to {out_lang_ui}. Use <sep> as a separator between the original transcript and the translation."
                )

        # O Processor do Phi-4 precisa de tokens de áudio no texto.
        # Tentamos com tokens comuns suportados: "<|audio|>" e "<|audio_1|>".
        AUDIO_PLACEHOLDERS = ["<|audio|>", "<|audio_1|>"]

        proc_inputs = None
        errors = []

        # Usa sempre o array (waveform) + sr; evita caminhos para reduzir ambiguidade
        for ph in AUDIO_PLACEHOLDERS:
            task_prompt = f"{ph}\n{base_instruction}"
            try:
                params = {"text": [task_prompt], "audios": [(audio_array, sr)]}
                proc_inputs = processor(return_tensors='pt', **params).to(DEVICE)
                break
            except Exception as e:
                errors.append(f"placeholder={ph}: {e}")
                continue

        if proc_inputs is None:
            return (
                "⚠️ Falha ao preparar inputs multimodais para geração.\n" +
                "\n".join(errors[:5])
            )

        # Geração mais rápida/estável
        try:
            model.eval()
        except Exception:
            pass
        try:
            with torch.inference_mode():
                gen_ids = model.generate(
                    **proc_inputs,
                    max_new_tokens=256,
                    do_sample=False,
                    temperature=0.0,
                    top_p=1.0,
                    num_beams=1,
                    early_stopping=True,
                    use_cache=False  # Evita DynamicCache.get_usable_length errors
                )
        except Exception as e:
            tb = traceback.format_exc(limit=2)
            return f"⚠️ Erro durante geração: {e}\n{tb}\nTenta um áudio mais curto (<= {max_seconds}s)."

        # Se existir input_ids, corta o prefixo do prompt
        try:
            cut = proc_inputs['input_ids'].shape[1]
            gen_ids = gen_ids[:, cut:]
        except Exception:
            pass

        text = _safe_decode_text(processor, gen_ids)

        # Se não for bengali, tenta extrair após <sep>
        if out_lang_ui.lower() != 'bengali':
            result = text.split('<sep>')[-1].strip() if '<sep>' in text else text
            return result

        # Tradução fallback para bengali (usa cache global)
        try:
            tok, m = get_m2m_models()
            lang_code_map = {'português': 'pt', 'espanhol': 'es', 'inglês': 'en'}
            src_code = lang_code_map.get(in_lang_ui, 'pt')
            tok.src_lang = src_code
            inputs_tx = tok(text, return_tensors='pt').to(DEVICE)
            out_ids = m.generate(**inputs_tx, forced_bos_token_id=tok.get_lang_id('bn'))
            translated = tok.batch_decode(out_ids, skip_special_tokens=True)[0]
            return translated
        except Exception as e:
            return f"(Transcrição)\n{text}\n\n⚠️ Fallback de tradução para bengali falhou: {e}"

    except Exception as e:
        # Captura qualquer erro não tratado e retorna texto detalhado para a UI
        return "⚠️ Erro inesperado na pipeline ASR:\n" + str(e) + "\n\n" + traceback.format_exc()


## 5) Interface Gradio e publicação via Cloudflared

In [ ]:
import gradio as gr

# WORKAROUND: Monkey-patch gradio_client para corrigir bug de schema parsing
try:
    from gradio_client import utils as gc_utils
    original_get_type = gc_utils.get_type if hasattr(gc_utils, 'get_type') else None
    
    if original_get_type:
        def patched_get_type(schema):
            """Wrapper que lida com schemas booleanos."""
            # Se schema for bool (True/False), retorna tipo genérico
            if isinstance(schema, bool):
                return "any"
            # Senão, chama função original
            return original_get_type(schema)
        
        gc_utils.get_type = patched_get_type
        print("✅ Patch aplicado ao gradio_client.utils.get_type")
except Exception as e:
    print(f"⚠️ Não foi possível aplicar patch: {e}")

INPUT_LANGS = ['português', 'espanhol', 'inglês']
OUTPUT_LANGS = ['português', 'espanhol', 'inglês', 'bengali']
CF_PUBLIC_URL = None


def run_pipeline(audio_path, in_lang_ui, out_lang_ui):
    try:
        if not audio_path:
            return 'Grava ou faz upload de um áudio primeiro 😉'
        return phi4_transcribe_or_translate(audio_path, in_lang_ui, out_lang_ui)
    except Exception as e:
        import traceback
        return "⚠️ Erro ao executar a pipeline:\n" + str(e) + "\n\n" + traceback.format_exc()


def get_public_url():
    global CF_PUBLIC_URL
    return CF_PUBLIC_URL or 'URL ainda não disponível: lança o túnel Cloudflared na célula seguinte.'

with gr.Blocks() as demo:
    gr.Markdown('## 🎙️ ASR + Tradução com **Phi-4-multimodal-instruct**')
    gr.Markdown('**Fala no microfone OU faz upload de um ficheiro de áudio** (MP3, WAV, etc.) e recebe o texto na língua selecionada.')
    gr.Markdown('📁 Nota: sem microfone no Colab, usa upload de ficheiro. Bengali é tradução de fallback após transcrição.')

    with gr.Row():
        in_lang = gr.Dropdown(choices=INPUT_LANGS, value='português', label='Língua de entrada (fala)')
        out_lang = gr.Dropdown(choices=OUTPUT_LANGS, value='português', label='Língua de saída (texto)')

    audio_input = gr.Audio(
        sources=['microphone', 'upload'],
        type='filepath',
        label='🎙️ Microfone ou Upload de Áudio (MP3/WAV)'
    )

    out_text = gr.Textbox(label='Texto de saída', interactive=False, lines=8)
    public_url_box = gr.Textbox(label='Public URL (Cloudflared)', interactive=False)

    btn = gr.Button('🚀 Transcrever / Traduzir')
    btn.click(run_pipeline, inputs=[audio_input, in_lang, out_lang], outputs=out_text)

    btn_show_url = gr.Button('🔗 Mostrar Public URL')
    btn_show_url.click(get_public_url, inputs=None, outputs=public_url_box)

    # Dica rápida na UI sobre limites e erros
    gr.Markdown('ℹ️ Dica: usa ficheiros ≤ 30s. Erros completos aparecem nesta caixa de texto.')

demo

## 6) Lançar a interface e expor com Cloudflared (TryCloudflare)

In [ ]:
import subprocess, re, time, shutil, os, signal, socket

# Fecha instância anterior para evitar relaunch e erros de SSE
try:
    demo.close()
except Exception:
    pass

# Escolhe uma porta livre dinamicamente

def is_port_free(port: int) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.connect_ex(("127.0.0.1", port)) != 0


def get_free_port(preferred: int | None = None) -> int:
    # Tenta usar porta do env se estiver livre
    if preferred and is_port_free(preferred):
        return preferred
    # Procura uma porta livre (ephemeral)
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]

env_port = None
try:
    env_port = int(os.environ.get("GRADIO_SERVER_PORT", ""))
except Exception:
    env_port = None

PORT = get_free_port(env_port or 7866)
os.environ["GRADIO_SERVER_PORT"] = str(PORT)
print(f"🔌 A usar porta livre: {PORT}")

# Lança Gradio sem queue; desativa share interno (usamos Cloudflared)
try:
    app = demo.launch(
        server_name='0.0.0.0',
        server_port=PORT,
        share=False,
        inbrowser=False,
        show_error=True,
        prevent_thread_lock=True,
    )
except ValueError as e:
    print("⚠️ Localhost indisponível para acesso direto. A criar link share do Gradio...")
    print(f"Detalhe: {e}")
    app = demo.launch(
        server_name='0.0.0.0',
        server_port=PORT,
        share=True,
        inbrowser=False,
        show_error=True,
        prevent_thread_lock=True,
    )

print(f"🌐 UI local: http://localhost:{PORT}")

# Pequeno atraso para o servidor arrancar
time.sleep(2)

# Termina túnel anterior se existir
if 'CF_PROC' in globals():
    try:
        if CF_PROC and CF_PROC.poll() is None:
            CF_PROC.terminate()
            try:
                CF_PROC.wait(timeout=5)
            except subprocess.TimeoutExpired:
                CF_PROC.kill()
    except Exception:
        pass

# Detecta binário do cloudflared (PATH ou ficheiro local ./cloudflared)

def ensure_cloudflared_path():
    path = shutil.which('cloudflared')
    if path:
        return path
    local = './cloudflared'
    if os.path.exists(local) and os.access(local, os.X_OK):
        return local
    return None

cf_bin = ensure_cloudflared_path()
if not cf_bin:
    print("⚠️ Cloudflared não encontrado. Usa a célula de setup para baixar (wget …) ou instala manualmente.")
else:
    # Inicia túnel para o servidor local
    CF_PROC = subprocess.Popen(
        [cf_bin, 'tunnel', '--url', f'http://localhost:{PORT}'],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True,
    )

    url_pattern = re.compile(r"(https://[a-z0-9-]+\.trycloudflare\.com)")
    public_url = None
    start = time.time()

    try:
        # Lê linhas do cloudflared até extrair o URL (timeout de 30s)
        while True:
            if CF_PROC.poll() is not None:
                break
            ln = CF_PROC.stdout.readline()
            if not ln:
                if time.time() - start > 30:
                    break
                time.sleep(0.1)
                continue
            m = url_pattern.search(ln)
            if m:
                public_url = m.group(1)
                # Atualiza variável global usada pelo botão "Mostrar Public URL"
                try:
                    CF_PUBLIC_URL
                except NameError:
                    CF_PUBLIC_URL = None
                CF_PUBLIC_URL = public_url
                print(f"✅ Public URL: {public_url}\n")
                print("Dica: usa o botão 'Mostrar Public URL' na UI para copiar.")
                break
    except Exception as e:
        print("⚠️ Erro a ler saída do cloudflared:", e)

    if not public_url:
        print('📁 Sem URL público. Verifica conectividade de rede do runtime.')
